In [1]:
# -*- coding:utf-8 -*-
from __future__ import absolute_import
from __future__ import print_function
import pandas as pd
import numpy as np
import h5py
#import math
import gensim

import jieba
jieba.enable_parallel(32)
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop, Adagrad
from keras.utils import plot_model
from keras.models import Sequential, load_model, model_from_json
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, GRU
from keras.layers.wrappers import Bidirectional
from keras.callbacks import TensorBoard
from keras.metrics import binary_accuracy
from keras import backend as K
from pickle import dump, load
import pymysql

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.187 seconds.
Prefix dict has been built succesfully.
Using TensorFlow backend.


In [2]:
def get_db_data(query_str):
        conn = pymysql.connect(host='127.0.0.1',port=3306,user='analyzer',password='analyzer@tbs2016',database='dp_relation',charset='utf8mb4',cursorclass=pymysql.cursors.DictCursor)
        cur=conn.cursor()
        doc = pd.read_sql_query(query_str, conn)
        doc = pd.DataFrame(doc)
        for column in doc.columns:
            column_null = pd.isnull(doc[column])
            column_null_true = column_null[column_null == True]
            if len(column_null_true) == len(doc):
                del doc[column]
        cur.close()
        conn.close()
        return doc

test_data = get_db_data("select * from weibo_raw_data where keyword_id='37_5'")
test_data.loc[0:10,'content']
comment = test_data

In [3]:
jieba.enable_parallel(32)
cw = lambda x: list(jieba.cut(str(x))) #定义分词函数
maxlen = 50
print('try loading pretrained word2vec model.')
wordvec_model = gensim.models.word2vec.Word2Vec.load('/home/da/nlp_repo/rnn_test/word2vec_wx/word2vec_wx')
wordvec_weight = wordvec_model.wv.syn0
vocab = dict([(k, v.index) for k, v in wordvec_model.wv.vocab.items()])
word_to_id = lambda word: not (vocab.get(word) is None) and vocab.get(word) or 0
words_to_ids = lambda words: list(map(word_to_id, words))
reverse_seq = lambda id_seq: id_seq[::-1]
# concat_seq = lambda df: np.concatenate((df.sent, df.sent_rev))
concat_seq = lambda a,b: list(np.hstack((a, b)))
comment = comment[comment['content'].notnull()] #仅读取非空评论
comment['words'] = comment['content'].apply(cw) #评论分词
comment['sent'] = comment['words'].apply(words_to_ids)
comment['sent_rev'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen, padding='pre', truncating='pre'))
comment['sent_rev'] = comment['sent_rev'].apply(reverse_seq)
comment['sent'] = list(sequence.pad_sequences(comment['sent'], maxlen=maxlen, padding='post', truncating='post'))
# print(comment['sent'].shape, comment['sent_rev'].shape)
comment['sent'] = comment['sent'].combine(comment['sent_rev'], func=concat_seq)
# print(comment['sent'][0])
# comment['sent'] = comment.apply(concat_seq, axis=1, raw=True, reduce=True)
comment

try loading pretrained word2vec model.


,id,project_id,project_name,keyword_id,keyword,user_id,content,created_at,url,comment_count,attitude_count,repost_count,source,words,sent,sent_rev
0,2077339,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,这么小的一个电阻 510欧！ ​,2017-04-09 17:45:00 +0800,http://weibo.com/2076419035/EDJ2F3kDF,0,2,0,Weibo.intl,"[这么, 小, 的, 一个, 电阻, , 510, 欧, ！, , ​]","[411, 121, 1, 35, 25284, 5, 30244, 3465, 18, 5...","[125790, 5, 18, 3465, 30244, 5, 25284, 35, 1, ..."
1,2077340,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我以后再也不相信女的不会秃顶这种屁话了。 ​,2017-04-09 13:50:48 +0800,http://weibo.com/2076419035/EDHvBb6hB,2,0,0,Weibo.intl,"[我, 以后, 再也, 不, 相信, 女, 的, 不会, 秃顶, 这种, 屁话, 了, 。,...","[20, 563, 2919, 19, 702, 807, 1, 236, 42043, 2...","[125790, 5, 3, 7, 98264, 241, 42043, 236, 1, 8..."
2,2077341,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,护肤然后害熬夜 是不是有病啊 ​,2017-04-09 04:01:35 +0800,http://weibo.com/2076419035/EDDEr4Iz5,1,1,0,Weibo.intl,"[护肤, 然后, 害, 熬夜, , 是不是, 有病, 啊, , ​]","[2426, 372, 5979, 4361, 5, 854, 0, 332, 5, 125...","[125790, 5, 332, 0, 854, 5, 4361, 5979, 372, 2..."
3,2077342,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,现在竟然还有这个 我感觉这个是卖烤冷面的大爷用来呲水的 ​,2017-04-08 12:39:10 +0800,http://weibo.com/2076419035/EDxC1wPvC,3,1,0,Weibo.intl,"[现在, 竟然, 还有, 这个, , 我, 感觉, 这个, 是, 卖, 烤, 冷面, 的,...","[199, 2191, 230, 101, 5, 20, 393, 101, 6, 1012...","[125790, 5, 1, 281, 44042, 2227, 8540, 1, 3041..."
4,2077343,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,他走得好慢，好驼，好弱。好苦。\n我看着他过马路，既为他担心，也觉得那样缓慢孤独被遗弃的命运...,2017-04-06 23:49:40 +0800,http://weibo.com/2076419035/EDj9cc4vm,0,3,0,Weibo.intl,"[他, 走, 得, 好慢, ，, 好, 驼, ，, 好弱, 。, 好苦, 。, \n, 我,...","[39, 261, 127, 0, 0, 58, 28550, 0, 0, 3, 0, 3,...","[125790, 5, 3, 379, 1, 2861, 42, 75, 0, 2833, ..."
5,2077344,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我估计是我天天想着为啥我这么倒霉然后我就倒霉了。以后我要天天高歌我真幸运！我真幸运！ ​,2017-04-04 17:31:54 +0800,http://weibo.com/2076419035/ECXORjMev,2,0,0,Weibo.intl,"[我, 估计, 是, 我, 天天, 想着, 为啥, 我, 这么, 倒霉, 然后, 我, 就,...","[20, 3642, 6, 20, 2026, 3980, 7967, 20, 411, 1...","[125790, 5, 18, 3337, 704, 20, 18, 3337, 704, ..."
6,2077345,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,为啥出租车要按公里算钱呢？按时间算是不能走的快点儿？？ ​,2017-04-04 17:18:57 +0800,http://weibo.com/2076419035/ECXJBnX1N,5,0,0,Weibo.intl,"[为啥, 出租车, 要, 按, 公里, 算钱, 呢, ？, 按, 时间, 算是, 不能, 走...","[7967, 5287, 31, 112, 1259, 0, 179, 36, 112, 1...","[125790, 5, 36, 36, 103617, 1, 261, 182, 3704,..."
7,2077346,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,东北一家人太好看了！！！大家快追起来！！ ​,2017-04-04 15:12:20 +0800,http://weibo.com/2076419035/ECWUd9Hxl,4,0,0,Weibo.intl,"[东北, 一家人, 太, 好看, 了, ！, ！, ！, 大家, 快追, 起来, ！, ！,...","[3184, 6477, 217, 2412, 7, 18, 18, 18, 151, 27...","[125790, 5, 18, 18, 311, 275007, 151, 18, 18, ..."
8,2077347,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我喜欢的表情的排名\n1号可爱表情：[熊猫]2:[兔子]3:[羞嗒嗒]4:[傻眼] ​,2017-04-04 00:40:31 +0800,http://weibo.com/2076419035/ECRcl7C59,4,0,0,Weibo.intl,"[我, 喜欢, 的, 表情, 的, 排名, \n, 1, 号, 可爱, 表情, ：, [, ...","[20, 233, 1, 3478, 1, 2431, 2, 44, 190, 2001, ...","[125790, 5, 1152, 35844, 1149, 47, 99, 1152, 6..."
9,2077348,37,美素化妆品_170411,37_5,倩碧_消费者用户微博原创,130788,我好朋友真的讲话都挺厉害的还。我记得我高中去看他辩论会的时候，同学A举起两张白纸说：假设这是...,2017-04-04 00:30:28 +0800,http://weibo.com/2076419035/ECR8ganjm,2,0,0,Weibo.intl,"[我, 好, 朋友, 真的, 讲话, 都, 挺, 厉害, 的, 还, 。, 我, 记得, 我...","[20, 58, 215, 474, 3287, 24, 1992, 3191, 1, 65...","[125790, 5, 3, 158, 78, 20, 6, 25185, 0, 11626..."


In [5]:
print('data ready. begin predicting.')
with open('/home/jeffmxh/rnn_test/models/2gru_seven_senti_0509_train.json', 'rt') as f:
    json_string=f.read()
model = model_from_json(json_string)
model.load_weights('/home/jeffmxh/rnn_test/models/2gru_seven_senti_0509_train_weight.h5')

pred_x = np.array(list(comment['sent']))
pred_y = model.predict(pred_x)
comment['mark']=list(pred_y)
comment.loc[0:5,['content','mark']]

data ready. begin predicting.


,content,mark
0,这么小的一个电阻 510欧！ ​,"[0.272601, 0.0540417, 0.237409, 0.263933, 0.07..."
1,我以后再也不相信女的不会秃顶这种屁话了。 ​,"[0.793809, 0.0776843, 0.0156825, 0.00710391, 0..."
2,护肤然后害熬夜 是不是有病啊 ​,"[0.265129, 0.18087, 0.00680298, 0.00491867, 0...."
3,现在竟然还有这个 我感觉这个是卖烤冷面的大爷用来呲水的 ​,"[0.436083, 0.0667314, 0.0709681, 0.0841312, 0...."
4,他走得好慢，好驼，好弱。好苦。\n我看着他过马路，既为他担心，也觉得那样缓慢孤独被遗弃的命运...,"[0.599542, 0.0124124, 0.0901273, 0.0307632, 0...."
5,我估计是我天天想着为啥我这么倒霉然后我就倒霉了。以后我要天天高歌我真幸运！我真幸运！ ​,"[0.178085, 0.0529095, 0.103157, 0.168485, 0.43..."


In [8]:
comment.loc[0,'mark']

array([ 0.53063953,  0.0062842 ,  0.11723289,  0.31447291,  0.0049616 ,
        0.02334569,  0.00196149,  0.00110171], dtype=float32)

In [6]:
rev_trans_dict = dict(zip(range(8), ['none', 'disgust', 'like', 'happiness', 'sadness', 'surprise', 'anger', 'fear']))

def result_translate(prob_list):
    sign_dict = dict(zip(prob_list, rev_trans_dict.values()))
    result_emotion_code = sign_dict[max(prob_list)]
    return result_emotion_code

comment['emotion_result'] = comment['mark'].apply(result_translate)
comment = comment.loc[:,['content','emotion_result']]
comment

,content,emotion_result
0,这么小的一个电阻 510欧！ ​,none
1,我以后再也不相信女的不会秃顶这种屁话了。 ​,none
2,护肤然后害熬夜 是不是有病啊 ​,sadness
3,现在竟然还有这个 我感觉这个是卖烤冷面的大爷用来呲水的 ​,none
4,他走得好慢，好驼，好弱。好苦。\n我看着他过马路，既为他担心，也觉得那样缓慢孤独被遗弃的命运...,none
5,我估计是我天天想着为啥我这么倒霉然后我就倒霉了。以后我要天天高歌我真幸运！我真幸运！ ​,sadness
6,为啥出租车要按公里算钱呢？按时间算是不能走的快点儿？？ ​,none
7,东北一家人太好看了！！！大家快追起来！！ ​,happiness
8,我喜欢的表情的排名\n1号可爱表情：[熊猫]2:[兔子]3:[羞嗒嗒]4:[傻眼] ​,like
9,我好朋友真的讲话都挺厉害的还。我记得我高中去看他辩论会的时候，同学A举起两张白纸说：假设这是...,none


In [11]:
print('save result to excel...')
writer = pd.ExcelWriter('37_5predict_single_emotion.xlsx')
comment.to_excel(writer, sheet_name='senti', encoding = 'utf-8', index = False)
writer.save()
print('task done.')

save result to excel...
task done.
